# The Energy Costs of Echolocation by Bats - An Observational Study

To orient themselves with respect to their surroundings, some bats use echolocation.  They send out pulses and read the echos that are bounced back from surrounding objects.  Such a trait has evolved in very few animal species, perhaps because of the high energy involved in producing pulses.  Because flight also requires a great deal of energy, zoologists wondered whether the combined energy costs of echolocation and flight in bats was the sum of the flight energy costs and the at-rest echolocation energy costs, or whether the bats had developed a means of echolocation in flight that made the combined energy cost less than the sum.

In [ ]:
# standard library imports

# 3rd party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

sns.set()

## Inspection and Graphical Display

In [ ]:
df = pd.read_csv('case1002.csv')
df.head()

In [ ]:
df.pivot_table(values=['Mass', 'Energy'], index='Type', aggfunc=['mean', 'std'])

There is strong evidence against the assumption of equal SDs.

In [ ]:
fig, ax = plt.subplots()
_ = sns.scatterplot(data=df, x='Mass', y='Energy', hue='Type', ax=ax)

The mass-energy relationship looks non-linear.  Try a log transform on both energy and mass.

In [ ]:
df['logmass'] = np.log(df['Mass'])
df['logenergy'] = np.log(df['Energy'])
fig, ax = plt.subplots()
_ = sns.scatterplot(data=df, x='logmass', y='logenergy', hue='Type', ax=ax)

In [ ]:
df.pivot_table(values=['logmass', 'logenergy'], index='Type', aggfunc=['mean', 'std'])

## Model

In [ ]:
formula = "np.log(Energy) ~ np.log(Mass) + C(Type, Treatment(reference='non-echolocating bats'))"
model = smf.ols(formula, data=df).fit()
model.summary()

## Do non-echolocating bats differ from echolocating bats?  Do non-echolocating bats differ from birds?

These questions are asking if the coefficients for echolocating bats and non-echolocating birds are zero.  From the summary above, there is weak evidence for a difference between non-echolocating bats and echolocating bats. ($t_{0.95, 16}$ = 0.388, $p$-value = 0.703).  There is also weak evidence for a difference between non-echolocating bats and birds ($t_{0.95, 16}$ = 0.896, $p$-value = 0.384).

## How much more energy is expended by echolocating bats?

Since we are looking at an additive model, the 95% confidence interval for the extra energy expected by echolocating bats is -0.351 to 0.508 log energy units, and our estimate is 0.0787 log energy units.  Back transforming out, we estimate that the median in-flight energy expediture of echolocating bats is $e^{0.0787} = 1.0819$ times that of non-echolocating bats.  We are 95% confident that the true median multiplicative factor is between $e^{-0.351} = 0.704$ and $e^{0.508} = 1.662$.

## How might other models compare?
### Is there an effect due to type?

$
\begin{align}
\mu\{\log(E)|\log(M), TYPE\} = \beta_0 + \beta_1 \cdot \log(M) + \beta_2 \cdot bird + \beta_3 \cdot ebat
\end{align}
$



$
\begin{align}
H_0: \: &\beta_2 = \beta_3 = 0 \\
H_a: \: &\text{at least one of the coefficients} \: \textemdash \: \beta_2 \: \text{or} \: \beta_3 \: \textemdash \: \text{is nonzero}
\end{align}
$

In [ ]:
full_model = smf.ols('np.log(Energy) ~ np.log(Mass) + C(Type, Treatment(reference="non-echolocating bats"))', data=df).fit()
reduced_model = smf.ols('np.log(Energy) ~ np.log(Mass)', data=df).fit()

In [ ]:
sm.stats.anova_lm(reduced_model, full_model)

There is only weak evidence of a lack of fit to the reduced model, i.e. there is only weak evidence for an effect due to body type after accounting for body mass ($F_{2,16}$ = 0.4278, $p$-value = 0.6593).